In [10]:
import os
import re
import json
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset

trial = "baselines/react_train_all"

# Configuration: adjust these paths as needed
METADATA_CSV = f"game_logs/{trial}/alfworld_results.csv"
PROMPT_BASE_DIR = f"game_logs/{trial}/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Something to include to fix unsloth on jupyter
os.environ["TRITON_DISABLE"] = "1"

In [2]:
# # --- config you can tweak ---
# CSV_PATH   = f"game_logs/{trial}/alfworld_results.csv"
# DATA_ROOT  = "/workspace/data/alfworld/json_2.1.1"
# SPLITS     = ["train", "valid_seen", "valid_unseen"]
# TOPK       = 50

# # weights for a simple, nav-free difficulty score
# W = {
#     "pickup": 1.0,
#     "put": 1.0,
#     "open": 0.6,
#     "close": 0.6,
#     "toggle": 1.2,     # faucets, stove, microwave, lamps…
#     "slice": 1.2,
#     "use": 1.0,
#     "uniq_objects": 1.2,
#     "uniq_recepts": 1.0,
#     "high_pddl_len": 1.0,
# }

# # -------------------------------------------------------------
# import json, pandas as pd
# from pathlib import Path
# from collections import defaultdict

# NAV_ACTIONS = {"MoveAhead","RotateLeft","RotateRight","LookUp","LookDown",
#                "MoveBack","MoveRight","MoveLeft"}

# API2CAT = {
#     "PickupObject":  "pickup",
#     "PutObject":     "put",
#     "OpenObject":    "open",
#     "CloseObject":   "close",
#     "ToggleObjectOn":  "toggle",
#     "ToggleObjectOff": "toggle",
#     "SliceObject":   "slice",
#     "UseObject":     "use",  # occasionally present
# }

# def resolve_traj(env_ref: str, data_root: Path, splits: list[str]) -> Path | None:
#     """
#     env_ref like: 'pick_cool_then_place_in_recep-.../trial_T2019...'
#     Returns the path to traj_data.json, or None.
#     """
#     p = data_root / env_ref
#     if p.is_dir() and (p / "traj_data.json").is_file():
#         return p / "traj_data.json"
#     if p.name == "traj_data.json" and p.is_file():
#         return p
#     if p.name == "game.tw-pddl" and p.with_name("traj_data.json").is_file():
#         return p.with_name("traj_data.json")
#     for sp in splits:
#         base = data_root / sp / env_ref
#         if base.is_dir() and (base / "traj_data.json").is_file():
#             return base / "traj_data.json"
#         if base.name == "traj_data.json" and base.is_file():
#             return base
#         if base.name == "game.tw-pddl" and base.with_name("traj_data.json").is_file():
#             return base.with_name("traj_data.json")
#     return None

# def difficulty_non_nav(traj_json: dict) -> dict:
#     """Compute nav-free difficulty signals from traj_data.json dict."""
#     low = traj_json["plan"]["low_actions"]
#     hi  = traj_json["plan"].get("high_pddl", [])

#     counts = defaultdict(int)
#     uniq_objs    = set()
#     uniq_recepts = set()

#     for step in low:
#         api = (step.get("api_action") or {})
#         a = api.get("action", "")
#         if a in NAV_ACTIONS:
#             continue  # drop pure nav

#         # map to interaction category (fallback to name if unknown)
#         cat = API2CAT.get(a)
#         if cat:
#             counts[cat] += 1

#         # track objects touched
#         oid = api.get("objectId")
#         rid = api.get("receptacleObjectId")
#         if oid: uniq_objs.add(oid)
#         if rid: uniq_recepts.add(rid)

#     # high-level (subgoal) count
#     high_len = len(hi)
    
#     # build score (tweak W as needed)
#     score = (
#         W["pickup"] * counts["pickup"] +
#         W["put"]    * counts["put"] +
#         W["open"]   * counts["open"] +
#         W["close"]  * counts["close"] +
#         W["toggle"] * counts["toggle"] +
#         W["slice"]  * counts["slice"] +
#         W["use"]    * counts["use"] +
#         W["uniq_objects"] * len(uniq_objs) +
#         W["uniq_recepts"] * len(uniq_recepts) +
#         W["high_pddl_len"] * high_len
#     )

#     return {
#         "score": round(float(score), 3),
#         "high_pddl_len": high_len,
#         "n_pickup": counts["pickup"],
#         "n_put": counts["put"],
#         "n_open": counts["open"],
#         "n_close": counts["close"],
#         "n_toggle": counts["toggle"],
#         "n_slice": counts["slice"],
#         "n_use": counts["use"],
#         "uniq_objects": len(uniq_objs),
#         "uniq_recepts": len(uniq_recepts),
#     }

# # ---------- run over your CSV and rank top-K per env_type ----------
# CSV = pd.read_csv(CSV_PATH)
# CSV = CSV[CSV["success"]==True]
# env_ref_col = "env_referencem" if "env_referencem" in CSV.columns else "env_reference"
# need = ["env_idx", "env_type", env_ref_col]
# CSV = CSV.dropna(subset=need)[need].drop_duplicates(subset=["env_idx"]).reset_index(drop=True)

# records = []  # (env_type, score, env_idx, traj_path, metrics_dict)
# data_root = Path(DATA_ROOT)
# skipped = 0

# for _, row in CSV.iterrows():
#     env_idx = int(row["env_idx"])
#     env_type = str(row["env_type"])
#     env_ref = str(row[env_ref_col])
#     tpath = resolve_traj(env_ref, data_root, SPLITS)
#     if not tpath:
#         skipped += 1
#         continue
#     try:
#         with open(tpath) as f:
#             tj = json.load(f)
#         m = difficulty_non_nav(tj)
#         records.append((env_type, m["score"], env_idx, str(tpath), m))
#     except Exception:
#         skipped += 1

# if skipped:
#     print(f"Note: skipped {skipped} unresolved/invalid trajectories.")

# # group and pick top-K
# by_type = defaultdict(list)
# for env_type, score, eidx, pth, m in records:
#     by_type[env_type].append((score, eidx, pth, m))

# topk_by_type = {}
# env_hardness = {}
# print(f"\nTop-{TOPK} hardest (nav-free) by env_type:\n")
# for t in sorted(by_type):
#     # tie-breaker: smaller env_idx first
#     topk = sorted(by_type[t], key=lambda x: (-x[0], x[1]))[:TOPK]
#     topk_by_type[t] = [eidx for score, eidx, _, _ in topk]
#     env_hardness.update({eidx:score for score, eidx, _, _ in topk})
#     print(f"[{t}]")
#     for rank, (score, eidx, pth, m) in enumerate(topk, 1):
#         print(f"  #{rank}  score={score:<6} env_idx={eidx:<5}  {pth}  "
#               f"(hi={m['high_pddl_len']}, pick={m['n_pickup']}, put={m['n_put']}, "
#               f"open={m['n_open']}, close={m['n_close']}, tog={m['n_toggle']}, "
#               f"obj={m['uniq_objects']}, rec={m['uniq_recepts']})")
#     print()

# print("env_idx lists:")
# for t in sorted(topk_by_type):
#     print(f'  {t:>8}: {topk_by_type[t]}')

# env_indices = []
# for key, value in topk_by_type.items():
#     print(key, value)
#     env_indices.extend(value)

In [3]:
# def peek_plan(traj_path):
#     j = json.load(open(traj_path))
#     plan = j.get("plan", {})
#     keys = list(plan.keys())
#     print("keys:", keys)
#     print("high_pddl len:", len(plan.get("high_pddl", [])))
#     print("high_actions len:", len(plan.get("high_actions", [])))
#     print("high_descs len:", len(plan.get("high_descs", [])))
#     print("low_actions len:", len(plan.get("low_actions", [])))
#     # optionally show a few items
#     print("sample high_actions:", plan.get("high_actions", [])[:5])
#     print("sample low_actions:", plan.get("low_actions", [])[:5])

# # Check experiment details
# traj = pathlib.Path("/rds/general/user/hai24/home/.cache/alfworld/json_2.1.1/train/pick_two_obj_and_place-CellPhone-None-SideTable-310/trial_T20190908_123211_754215/traj_data.json")
# peek_plan(traj)

In [4]:
# import json
# from pathlib import Path

# def print_high_plan(traj_path: str | Path):
#     p = Path(traj_path)
#     j = json.load(open(p))
#     plan = j.get("plan", {})

#     # Prefer high_pddl, fallback to high_actions
#     high = plan.get("high_pddl")
#     if high is None:
#         high = plan.get("high_actions")

#     if not high:
#         print(f"No high-level plan found in {p}")
#         print("Available keys under plan:", list(plan.keys()))
#         return

#     print(f"\nHigh-level plan from: {p}")
#     for i, step in enumerate(high, 1):
#         if isinstance(step, dict):
#             # print(step)
#             # common shapes: {"action":"OpenObject", "args":["fridge 1"]} or similar
#             act = step.get("discrete_action") or step.get("op") or step.get("name") or "UNKNOWN"
#             args = step.get("args") or step.get("arguments") or []
#             if isinstance(args, (list, tuple)):
#                 arg_str = ", ".join(map(str, args))
#             else:
#                 arg_str = str(args)
#             print(f"{i:02d}. {act}({arg_str})")
#         else:
#             # string step
#             print(f"{i:02d}. {step}")

# # Example usage:
# print_high_plan(traj)

In [5]:
def remove_few_shot_examples(prompt: str) -> str:
    """
    Keeps the first line ('Interact with a household...') and removes the few-shot examples
    between 'Here are 2 examples:' and the last 'Your task is to:'.
    """
    # Extract the first line (system instruction)
    first_line, *rest = prompt.split("\n", 1)
    rest_text = rest[0] if rest else ""

    #########################################

    # Split by all occurrences of 'Your task is to:'
    parts = rest_text.split("Your task is to:")
    if len(parts) < 2:
        return prompt  # Nothing to strip

    # Keep only the last task and everything that follows
    cleaned = first_line.strip() + "\n\nYour task is to:" + parts[-1]
    

    #########################################

    
    # # Split by all occurrences of 'Your task is to:'
    # parts = rest_text.split("Here is the task")
    # if len(parts) < 2:
    #     return prompt  # Nothing to strip

    # # Keep only the last task and everything that follows
    # cleaned = first_line.strip() + "\n\nHere is the task" + parts[-1]

    
    #########################################

    
    # parts = rest_text.split("Apply these rules silently to choose the next action.")
    # if len(parts) < 2:
    #     return prompt  # Nothing to strip
    # cleaned = first_line.strip() + "\n\nHere are some hints" + parts[-1]

    
    return cleaned.strip()

In [6]:
# env_indices = []
# for key, value in topk_by_type.items():
#     print(key, value)
#     env_indices.extend(value)

In [7]:
# df = pd.read_csv(METADATA_CSV)
# df = df[df["model"] == "Qwen/Qwen2.5-14B-Instruct"]
# df = df[df["prompt_name"] == "stringstate-1_0-k-goal+locations_visited+current_location+current_inventory+thought+action"]

# print(f"Average number of steps: {df.num_of_steps.mean()}")

# # ------------------------------------------------------------------
# # 4.  Keep *only* those rows whose env_idx is in that list
# df = df[df["env_idx"].isin(env_indices)].reset_index(drop=True)
# df = df[df["success"] == True].reset_index(drop=True)
# df["pddl_difficulty"] = df["env_idx"].map(env_hardness)

# def calc_successes(df):
#     # Group by env_type
#     grouped = df.groupby("env_type")["success"]
    
#     # Compute success rate and count
#     success_stats = grouped.agg(
#         success_rate=lambda x: 100 * x.mean(),
#         count="count"
#     ).round(2)
    
#     # Print
#     print("Success Rate and Count by env_type (before filtering):")
#     print(success_stats.to_string())
    
# def extract_whole_trace(example):
#     trace = json.load(open(example["trace_file"]))

#     # Prompt = system + initial user
#     sys_msg  = next(m["content"] for m in trace if m["role"]=="system")
#     usr_msg  = next(m["content"] for m in trace if m["role"]=="user")

#     # print(usr_msg)
#     prompt   = sys_msg + "\n" + usr_msg + "\n"
#     # Completion = *everything else*, in order
#     rest     = trace[2:]
#     completion = "".join(m["content"] + "\n" for m in rest)
#     # add end‐of‐sequence token if needed
#     if not completion.endswith(""):
#         completion += ""
        
#     match = re.findall("Nothing happens.", completion)
#     # if len(match) > 0:
#     #     print(len(match))
    
#     return {"prompt": prompt, "completion": completion, "nothing_occ":len(match)}


# print(f"Kept {len(df)} rows that match the difficult env_idx list.")
# print(df[["env_idx","env_type","success", "num_of_steps"]])
# df[["prompt", "completion", "nothing_occ"]] = df.apply(lambda row: pd.Series(extract_whole_trace(row)), axis=1)

# removed_examples = len(df[df.nothing_occ != 0])
# df = df[df.nothing_occ == 0]
# df.reset_index(drop=True, inplace=True)

# print("\n"+"*"*50 + f"\nTotal removed examples (with nothing happened): {removed_examples}\n" + "*"*50)

# TOPK = 20
# # df = df.groupby("env_type", group_keys=False).apply(lambda x: x.sample(TOPK, random_state=42))
# df = (
#     df.sort_values(['env_type','pddl_difficulty','env_idx'],
#                    ascending=[True, False, True])
#       .groupby('env_type', group_keys=False)
#       .head(TOPK)
# )
# df.reset_index(drop=True, inplace=True)

# calc_successes(df)

# # df = df[["prompt","completion","success","total_reward"]]

In [17]:
# This CSV must contain columns 'prompt_file', 'trace_file', and any other metadata fields you care about.
df = pd.read_csv(METADATA_CSV)
# df = df[df["model"] == "Qwen/Qwen2.5-14B-Instruct"]
# df = df[df["prompt_name"] == "stringstate-1_0-k-goal+locations_visited+current_location+current_inventory+thought+action"]

print(df.shape)
# display(df.head(5))


def calc_successes(df):
    # Group by env_type
    grouped = df.groupby("env_type")["success"]
    
    # Compute success rate and count
    success_stats = grouped.agg(
        success_rate=lambda x: 100 * x.mean(),
        count="count"
    ).round(2)
    
    # Print
    print("Success Rate and Count by env_type (before filtering):")
    print(success_stats.to_string())

calc_successes(df)

# 4. Filter out failed trials or include all
df = df[df["success"] == True].reset_index(drop=True)

# 5. Create a Hugging Face Dataset
# Drop the original file path columns if not needed
# columns_to_drop = ["prompt_file", "trace_file"]


def extract_whole_trace(example):
    # print(example["trace_file"])
    filename = os.path.basename(example["trace_file"])
    # print(filename)
    trace = json.load(open(PROMPT_BASE_DIR + filename))

    # Prompt = system + initial user
    sys_msg  = next(m["content"] for m in trace if m["role"]=="system")
    usr_msg  = next(m["content"] for m in trace if m["role"]=="user")

    # print(usr_msg)
    prompt   = sys_msg + "\n" + usr_msg + "\n"
    # Completion = *everything else*, in order
    rest     = trace[2:]
    completion = "".join(m["content"] + "\n" for m in rest)
    # add end‐of‐sequence token if needed
    if not completion.endswith(""):
        completion += ""
        
    match = re.findall("Nothing happens.", completion)
    # if len(match) > 0:
    #     print(len(match))
    
    return {"prompt": prompt, "completion": completion, "nothing_occ":len(match)}

def extract_final_trace(example, eos: str = ""):
    """
    Build:
      prompt     = system + first user
      completion = last assistant message ONLY
    If the last assistant has no 'content', fall back to 'old_content'.
    """
    with open(example["trace_file"], "r") as f:
        trace = json.load(f)

    # Prompt = first system + first user
    sys_msg = next((m.get("content", "") for m in trace if m.get("role") == "system"), "")
    usr_msg = next((m.get("content", "") for m in trace if m.get("role") == "user"), "")
    prompt  = (sys_msg + "\n" + usr_msg).rstrip() + "\n"

    # Completion = only the final assistant message
    last_assistant = next((m for m in reversed(trace) if m.get("role") == "assistant"), None)

    completion = last_assistant.get("content") or last_assistant.get("old_content") or ""
    completion = completion.rstrip() + "\n"

    # Optional EOS token
    if eos and not completion.endswith(eos):
        completion += eos

    return {"prompt": prompt, "completion": completion}

# apply to your pandas DataFrame before converting to a HF Dataset:
df[["prompt", "completion", "nothing_occ"]] = df.apply(lambda row: pd.Series(extract_whole_trace(row)), axis=1)

# removed_examples = len(df[df.nothing_occ != 0])
# df = df[df.nothing_occ == 0]

removed_examples = len(df[df.nothing_occ >= 2])
df = df[df.nothing_occ < 2]

df.reset_index(drop=True, inplace=True)
print("\n"+"*"*50 + f"\nTotal removed examples (with nothing happened): {removed_examples}\n" + "*"*50)
# df[["prompt", "completion"]] = df.apply(lambda row: pd.Series(extract_final_trace(row)), axis=1)

calc_successes(df)

print("\n"+"*"*50 + f"\nFinal Dataset Size {len(df)}\n" + "*"*50)

(1200, 32)
Success Rate and Count by env_type (before filtering):
          success_rate  count
env_type                     
clean            30.00    220
cool             65.80    193
examine          84.76    105
heat             44.79    163
put              87.80    254
puttwo           58.87    265

**************************************************
Total removed examples (with nothing happened): 151
**************************************************
Success Rate and Count by env_type (before filtering):
          success_rate  count
env_type                     
clean            100.0     58
cool             100.0    114
examine          100.0     56
heat             100.0     63
put              100.0    172
puttwo           100.0    120

**************************************************
Final Dataset Size 583
**************************************************


In [18]:
prompt = df.prompt[0]
print(prompt)

Interact with a household to solve a task.



Here are 2 examples:

You are in the middle of a room. Looking quickly around you, you see a cabinet 13, a cabinet 12, a cabinet 11, a cabinet 10, a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 1, a diningtable 1, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.
Your task is to: put a hot apple in fridge.
> think: To solve the task, I need to find and take an apple, then heat it with microwave, then put it in fridge.
OK.
> think: First I need to find an apple. An apple is more likely to appear in fridge (1), diningtable (1), coffeetable (1), drawer (1), cabinet (1-13), garbagecan (1). I can check one by one, starting with fridge 1.
OK.
> go to fridge 1
The fridge 1 is closed.
> open fridge 1
You open the

In [44]:
df = df[df["env_type"].isin(["puttwo", "examine"])]
df.reset_index(drop=True, inplace=True)
# df = df[df["num_of_steps"]>=df.num_of_steps.mean()]
# df.reset_index(drop=True, inplace=True)
print(len(df))
df.head()

193


,env_idx,env_type,agent_type,llm_type,model,temperature,prompt_name,success,done,total_reward,...,resample,resample_temperature,keys_to_use,additional_prompt_annotation,trace_file,prompt_file,env_reference,prompt,completion,nothing_occ
0,5,examine,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,look_at_obj_in_light-KeyChain-None-DeskLamp-30...,Interact with a household to solve a task.\n\n...,>goal: look at keychain under the desklamp\nlo...,0
1,401,puttwo,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_two_obj_and_place-CreditCard-None-SideTab...,Interact with a household to solve a task.\n\n...,>goal: put two creditcards in sidetable\nlocat...,0
2,7,puttwo,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_two_obj_and_place-Watch-None-SideTable-21...,Interact with a household to solve a task.\n\n...,>goal: find two watches and put them in sideta...,0
3,8,puttwo,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_two_obj_and_place-Pillow-None-Ottoman-208...,Interact with a household to solve a task.\n\n...,>goal: find two pillows and put them in ottoma...,0
4,11,puttwo,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_two_obj_and_place-Laptop-None-Bed-311/tri...,Interact with a household to solve a task.\n\n...,>goal: put two laptops in bed\nlocations visit...,0


In [56]:
print(df.nothing_occ[:120].mean())
df.nothing_occ[:120].value_counts().sort_index()

0.0


nothing_occ
0    120
Name: count, dtype: int64

In [57]:
print(df.nothing_occ.mean())
df.nothing_occ.value_counts().sort_index()

0.0


nothing_occ
0    781
Name: count, dtype: int64

In [9]:
df.prompt[0]

'Interact with a household to solve a task.\n\n\n============\nApply these rules silently to choose the next action.\nNever repeat, quote, or paraphrase this block in thought or action.\nIf any rule conflicts with the current observation, prefer the observation.\n\n=========\nHere are some hints:\n- Prioritize searching areas where the {object} is likely to be found.\n- Verify the presence of the {object} in the environment before proceeding.\n- Prioritize searching locations likely to contain the {object} needed for the task.\n=========\n\n============\n\nHere are 2 examples:\n\nYou are in the middle of a room. Looking quickly around you, you see a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 1, a diningtable 3, a diningtable 2, a diningtable 1, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a sidetable 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: clean some apple a

In [15]:
print("Number of Steps Statistics\n***************************")
print(f"""Min {df.num_of_steps.min()}
25th {df.num_of_steps.quantile(.25)}
Median {df.num_of_steps.median()}
Mean {df.num_of_steps.mean()}
75th {df.num_of_steps.quantile(.75)}
Max {df.num_of_steps.max()}
""")
# df.num_of_steps.median()

Number of Steps Statistics
***************************
Min 4
25th 7.0
Median 9.0
Mean 11.125
75th 13.0
Max 31



In [4]:
print(len(df))
df.head()

877


,env_idx,env_type,agent_type,llm_type,model,temperature,prompt_name,success,done,total_reward,...,total_history_token,correction,resample,resample_temperature,keys_to_use,additional_prompt_annotation,trace_file,prompt_file,prompt,completion
0,1,heat,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,2999,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: put a hot potato in garbagecan\nlocatio...
1,2,put,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,3402,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: put a spraybottle in cabinet\nlocations...
2,3,clean,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,3480,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: put a clean mug in coffeemachine\nlocat...
3,4,clean,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,3944,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: clean some spoon and put it in diningta...
4,5,examine,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,3728,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: look at keychain under the desklamp\nlo...


In [9]:
df = df[df["env_type"].isin(["puttwo", "examine"])]
df.reset_index(drop=True, inplace=True)
print(len(df))
df.head()

193


,env_idx,env_type,agent_type,llm_type,model,temperature,prompt_name,success,done,total_reward,...,total_history_token,correction,resample,resample_temperature,keys_to_use,additional_prompt_annotation,trace_file,prompt_file,prompt,completion
0,5,examine,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,3728,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: look at keychain under the desklamp\nlo...
1,7,puttwo,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,3823,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: find two watches and put them in sideta...
2,8,puttwo,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,3126,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: find two pillows and put them in ottoma...
3,11,puttwo,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,3079,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: put two laptops in bed\nlocations visit...
4,401,puttwo,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,4566,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,Interact with a household to solve a task.\n\n...,>goal: put two creditcards in sidetable\nlocat...


In [6]:
df = df[df["num_of_steps"]>=df.num_of_steps.mean()]
df.reset_index(drop=True, inplace=True)
print(len(df))
df.head()

calc_successes(df)

# df = df[df["num_of_steps"]<=df.num_of_steps.quantile(.25)]
# df.reset_index(drop=True, inplace=True)
# print(len(df))
# df.head()

255
Success Rate and Count by env_type (before filtering):
          success_rate  count
env_type                     
clean            100.0     61
cool             100.0     23
examine          100.0     19
heat             100.0     17
put              100.0     66
puttwo           100.0     69


In [15]:
q75 = df.num_of_steps.quantile(.75)
df = df[df["num_of_steps"]>=df.num_of_steps.mean()]
df = df[df["num_of_steps"]<=q75]

df.reset_index(drop=True, inplace=True)
print(len(df))
df.head()

102


,env_idx,env_type,agent_type,llm_type,model,temperature,prompt_name,success,done,total_reward,...,correction,resample,resample_temperature,keys_to_use,additional_prompt_annotation,trace_file,prompt_file,env_reference,prompt,completion
0,414,put,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_and_place_simple-CD-None-DiningTable-311/...,Interact with a household to solve a task.\n\n...,>goal: put a cd in diningtable\nlocations visi...
1,25,put,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_and_place_simple-SoapBottle-None-Cart-430...,Interact with a household to solve a task.\n\n...,>goal: put some soapbottle on cart\nlocations ...
2,808,put,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_and_place_simple-Pen-None-Shelf-319/trial...,Interact with a household to solve a task.\n\n...,>goal: put a pen in shelf\nlocations visited: ...
3,810,puttwo,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_two_obj_and_place-Pen-None-Drawer-311/tri...,Interact with a household to solve a task.\n\n...,>goal: put two pens in drawer\nlocations visit...
4,815,examine,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,look_at_obj_in_light-CD-None-DeskLamp-305/tria...,Interact with a household to solve a task.\n\n...,>goal: examine the cd with the desklamp\nlocat...


In [26]:
df.env_type.value_counts()

env_type
put        86
puttwo     69
clean      55
examine    29
cool       25
heat       25
Name: count, dtype: int64

In [ ]:
print("Number of Steps Statistics\n***************************")
print(f"""Min {df.num_of_steps.min()}
25th {df.num_of_steps.quantile(.25)}
Median {df.num_of_steps.median()}
Mean {df.num_of_steps.mean()}
75th {df.num_of_steps.quantile(.75)}
Max {df.num_of_steps.max()}
""")
# df.num_of_steps.median()

In [19]:
# Apply to entire DataFrame
df["prompt"] = df["prompt"].apply(remove_few_shot_examples)

In [20]:
df.prompt[0]

'Interact with a household to solve a task.\n\nYour task is to: put a hot potato in garbagecan.'

In [7]:
df.iloc[:10].num_of_steps

0    19
1    21
2    15
3     7
4    15
5     9
6    13
7    11
8    30
9    15
Name: num_of_steps, dtype: int64

In [21]:
# dataset = Dataset.from_pandas(df)
dataset = Dataset.from_pandas(df.drop(columns=["success", "total_reward"]))

In [22]:
dataset

Dataset({
    features: ['env_idx', 'env_type', 'agent_type', 'llm_type', 'model', 'temperature', 'prompt_name', 'done', 'early_stop', 'error', 'num_of_steps', 'num_nothing_happens', 'num_repetitions', 'num_no_command', 'num_no_json', 'num_correction', 'num_resample', 'total_prompt_token', 'total_in_token_accumulated', 'total_in_token_message_accumulated', 'total_out_token_accumulated', 'total_history_token', 'correction', 'resample', 'resample_temperature', 'keys_to_use', 'additional_prompt_annotation', 'trace_file', 'prompt_file', 'env_reference', 'prompt', 'completion', 'nothing_occ'],
    num_rows: 583
})

In [10]:
df.prompt[0]

'Interact with a household to solve a task.\n\nYour task is to: put a hot potato in garbagecan.'

In [11]:
df.head()

,env_idx,env_type,agent_type,llm_type,model,temperature,prompt_name,success,done,total_reward,...,correction,resample,resample_temperature,keys_to_use,additional_prompt_annotation,trace_file,prompt_file,env_reference,prompt,completion
0,1,heat,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_heat_then_place_in_recep-Potato-None-Garb...,Interact with a household to solve a task.\n\n...,>goal: put a hot potato in garbagecan\nlocatio...
1,2,put,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_and_place_simple-SprayBottle-None-Cabinet...,Interact with a household to solve a task.\n\n...,>goal: put a spraybottle in cabinet\nlocations...
2,403,put,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_and_place_simple-Pan-None-CounterTop-8/tr...,Interact with a household to solve a task.\n\n...,>goal: put a pan in countertop\nlocations visi...
3,10,cool,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_cool_then_place_in_recep-Plate-None-Cabin...,Interact with a household to solve a task.\n\n...,>goal: cool some plate and put it in cabinet\n...
4,410,cool,ours-text,VLLMChat,Qwen/Qwen2.5-14B-Instruct,0.0,stringstate-1_0-k-goal+locations_visited+curre...,True,True,1,...,True,False,NaN,"['goal', 'locations_visited', 'current_locatio...",NaN,game_logs/alfworld_eval_baseline_state_train_a...,game_logs/alfworld_eval_baseline_state_train_a...,pick_cool_then_place_in_recep-Bread-None-Micro...,Interact with a household to solve a task.\n\n...,>goal: put a cool bread in microwave\nlocation...


In [11]:
print("Number of Steps Statistics\n***************************")
print(f"""Min {df.num_of_steps.min()}
25th {df.num_of_steps.quantile(.25)}
Median {df.num_of_steps.median()}
Mean {df.num_of_steps.mean()}
75th {df.num_of_steps.quantile(.75)}
Max {df.num_of_steps.max()}
""")
# df.num_of_steps.median()

Number of Steps Statistics
***************************
Min 11
25th 13.0
Median 16.0
Mean 17.70980392156863
75th 21.0
Max 45



In [8]:
# 6. (Optional) Split into train/validation
# Here we do an 90/10 split; adjust 'seed' for reproducibility
dataset = dataset.train_test_split(test_size=0.1, seed=42)

train_ds = dataset["train"]
val_ds   = dataset["test"]


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['env_idx', 'env_type', 'agent_type', 'llm_type', 'model', 'temperature', 'prompt_name', 'done', 'early_stop', 'error', 'num_of_steps', 'num_nothing_happens', 'num_repetitions', 'num_no_command', 'num_no_json', 'num_correction', 'num_resample', 'total_prompt_token', 'total_in_token_accumulated', 'total_in_token_message_accumulated', 'total_out_token_accumulated', 'total_history_token', 'correction', 'resample', 'resample_temperature', 'keys_to_use', 'additional_prompt_annotation', 'trace_file', 'prompt_file', 'env_reference', 'prompt', 'completion', 'nothing_occ'],
        num_rows: 681
    })
    test: Dataset({
        features: ['env_idx', 'env_type', 'agent_type', 'llm_type', 'model', 'temperature', 'prompt_name', 'done', 'early_stop', 'error', 'num_of_steps', 'num_nothing_happens', 'num_repetitions', 'num_no_command', 'num_no_json', 'num_correction', 'num_resample', 'total_prompt_token', 'total_in_token_accumulated', 'total_in_t

In [9]:
# 7. Save to disk for later fine-tuning
OUT_DIR = "hf_alfworld_dataset/vanilla_stateact_train1200"
# OUT_DIR = "hf_alfworld_dataset/hints"
train_ds.save_to_disk(os.path.join(OUT_DIR, "train"))
val_ds.save_to_disk(os.path.join(OUT_DIR, "validation"))

print(f"Saved dataset to {OUT_DIR}")

Saving the dataset (0/1 shards):   0%|          | 0/789 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/88 [00:00<?, ? examples/s]

Saved dataset to hf_alfworld_dataset/vanilla_stateact_train1200


In [9]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import train_on_responses_only

def get_model_and_tokenizer(
        model_name="Qwen/Qwen2.5-14B-Instruct", 
        max_seq_length=1024,
        load_in_4bit=True, 
        gpu_memory_utilization=0.9,
        lora_rank=64,
        random_state=1234,
        get_peft=True,
        # lora_alpha=64
    ):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name ,
        max_seq_length = max_seq_length,
        load_in_4bit = load_in_4bit, # False for LoRA 16bit
        load_in_8bit = False,    # A bit more accurate, uses 2x memory
        full_finetuning = False, # We have full finetuning now!
        fast_inference = False, # Enable vLLM fast inference
        max_lora_rank = lora_rank,
        gpu_memory_utilization = gpu_memory_utilization, # Reduce if out of memory
        # device_map="cuda" ,               # or {"": 0} to pin to GPU 0
        device_map="auto",               # pin everything to GPU 0
        # trust_remote_code=True,
    )

    if get_peft:
        model = FastLanguageModel.get_peft_model(
            model,
            r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
            target_modules = [
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj",
            ], # Remove QKVO if out of memory
            # lora_alpha = lora_rank,
            lora_alpha = lora_rank*2,
            
            lora_dropout = 0.10,                    # add dropout (try 0.1–0.2)
            
            use_gradient_checkpointing = "unsloth", # Enable long context finetuning
            random_state = random_state,
        )
    
    return model, tokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 09-01 10:04:46 [__init__.py:235] Automatically detected platform cuda.


In [10]:
from trl import SFTTrainer, SFTConfig
from transformers import DataCollatorForSeq2Seq

def get_sft_trainer(model, tokenizer, dataset, eval_dataset=None, training_steps=10, collator=None):
    training_args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        # warmup_steps = 5,
        # warmup_steps = 10,
        warmup_ratio = 0.1,

        # label_smoothing_factor=0.1,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = training_steps,
        
        learning_rate = 2e-4,
        # learning_rate = 2e-5, # use for full train runs
        logging_steps = 1,
        optim = "adamw_8bit",
        
        weight_decay = 0.01,
        # weight_decay = 0.05, #increase to maybe improve
        
        lr_scheduler_type = "linear",
        # lr_scheduler_type = "cosine",
        seed = 42,
        
        # eval_strategy="steps",
        # eval_steps=3,                      # <-- frequency
        
        # report_to = "none", # Use this for WandB etc
    )
    
    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = dataset,
        eval_dataset = eval_dataset, # Can set up evaluation!
        args = training_args,
        data_collator = collator,
    )
    return trainer

In [10]:
import torch
import gc

try:
    del model
    del tokenizer
except:
    pass

torch.cuda.empty_cache()
gc.collect()


200

In [11]:
# model, tokenizer = get_model_and_tokenizer(max_seq_length=8192)
# model, tokenizer = get_model_and_tokenizer(model_name="Qwen/Qwen2.5-14B-Instruct")
model, tokenizer = get_model_and_tokenizer(model_name="Qwen/Qwen2.5-7B-Instruct")

==((====))==  Unsloth 2025.8.1: Fast Qwen2 patching. Transformers: 4.55.0. vLLM: 0.10.0.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.254 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.05it/s]
Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.8.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [23]:
# Prepare HF datasets
def merge_prompt_completion(example):
    # Make sure your completion already ends with EOS
    return {"text": example["prompt"] + example["completion"]}

try:
    hf_ds = dataset.map(
    merge_prompt_completion,
    remove_columns=[c for c in dataset["train"].column_names if c not in ("text",)],
    )
except:
    hf_ds = dataset.map(
        merge_prompt_completion,
        remove_columns=[c for c in dataset.column_names if c not in ("text",)],
    )



collator = None

Map: 100%|██████████| 583/583 [00:00<00:00, 3992.62 examples/s]


In [1]:
# # Check how many samples are truncated.
# i = 0
# lost_tokens = 0
# total_tokens = 0
# for text in hf_ds["text"]:
#     tokens = tokenizer.encode(text)
#     total_tokens += len(tokens)
#     if len(tokens) > 1024:
#         i += 1
#         lost_tokens += len(tokens)-1024
# print(f"Number of samples truncated: ({i/len(hf_ds)*100:.2f}%) {i}/{len(hf_ds)}")
# print(f"Total tokens truncated: ({lost_tokens/total_tokens*100:.2f}%) {lost_tokens}")

In [13]:
print("\n\n================\TRAINING SFT")

try:
    sft_trainer = get_sft_trainer(model, tokenizer, hf_ds["train"], eval_dataset=hf_ds["test"], training_steps=15, collator=collator)
except:
    sft_trainer = get_sft_trainer(model, tokenizer, hf_ds, training_steps=15, collator=collator)



================\TRAINING SFT


Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 707/707 [00:01<00:00, 598.86 examples/s]


In [14]:
sft_trainer.train()
print("\n\n\n\nDone.")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 707 | Num Epochs = 1 | Total steps = 89
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 161,480,704 of 7,777,097,216 (2.08% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.406900
2,1.359100
3,1.163400
4,0.910000
5,0.776700
6,0.681500
7,0.659300
8,0.595800
9,0.457900
10,0.328800






Done.


In [15]:
# wherever you want to save:

# out_dir = "models/14B_rag_20handpicked_lorareg" 
# out_dir = "models/14B_rag_all_lorareg" 
# out_dir = "models/14B_rag_all_lorareg_lowlr" 

# out_dir = "models/7B_vanilla_all" 
# out_dir = "models/7B_vanilla_20handpicked"

# out_dir = "models/7B_rag_all"
# out_dir = "models/7B_rag_20handpicked"

# out_dir = "models/14B_react_vanilla_all" 
# out_dir = "models/14B_react_rag_all" 
# out_dir = "models/14B_react_vanilla_20handpicked" 

# out_dir = "models/14B_state_rag_selfgenerated" 
# out_dir = "models/14B_react_rag_selfgenerated" 

# out_dir = "models/7B_state_rag_selfgenerated" 
out_dir = "models/7B_react_rag_selfgenerated" 

# write model + tokenizer
model.save_pretrained(out_dir)
tokenizer.save_pretrained(out_dir)
sft_trainer.save_model(out_dir)               # saves model, config, trainer state